In [1]:
# optimization to save on shuffle space
spark.conf.set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")

# this needs to be set to a reasonable number on a local machine
spark.conf.set("spark.sql.shuffle.partitions", "16")

spark.conf.set("spark.sql.execution.arrow.enabled", "true")

# hack to provide a posix like path in Windows
from pathlib import Path
checkpoint_dir = "file:///" + "/".join(
    (Path("..").resolve() / "data/tmp").parts[1:]
)
print(checkpoint_dir)

# needed by GraphFrames to persist intermediate results to local disk
sc.setCheckpointDir(checkpoint_dir)

file:///Users/acmiy/OneDrive/Documents/cs229/cs229-f19-wiki-forecast/data/tmp


In [3]:
import pyspark.sql.functions as F
from graphframes import GraphFrame

pages_path = "../data/enwiki/pages"
pagelinks_path = "../data/enwiki/pagelinks"
sampling_ratio = 0.02

pages = spark.read.parquet(pages_path)
pagelinks = spark.read.parquet(pagelinks_path)

def sample_graph(pages, pagelinks, sampling_ratio, relabel=True):
    vertices = (
        pages
        .sample(sampling_ratio)
        .withColumn("rank", F.expr("row_number() over (order by id)"))
        # ensure 0 index for mapping into a scipy.sparse matrix
        .withColumn("rank", F.expr("rank - 1"))
    )
    edges = (
        pagelinks.join(
            vertices.selectExpr("id as from", "rank as relabeled_src"),
            on="from", 
            how="inner"
        ).join(
            vertices.selectExpr("id as dest", "rank as relabeled_dst"), 
            on="dest", 
            how="inner"
        )
    ).selectExpr("from as src", "dest as dst", "relabeled_src", "relabeled_dst")
    
    if relabel:
        vertices = vertices.selectExpr("rank as id", "id as article_id")
        edges = edges.selectExpr("relabeled_src as src", "relabeled_dst as dst")
    return GraphFrame(vertices, edges)

### Find the largest connected component

The graph must be fully connected to apply spectral methods.

In [27]:
def connected_components(pages, pagelinks, sample_ratio):
    graph = sample_graph(pages, pagelinks, sample_ratio)
    total = graph.vertices.count()
    results = (
        graph.connectedComponents()
        .groupBy("component")
        .count()
        .orderBy(F.desc("count"))
        .withColumn("total", F.lit(total))
        .withColumn("frac", F.expr("round(count/total*100, 3)"))
    )
    results.show(n=5)

In [28]:
%time connected_components(pages, pagelinks, 0.02)
%time connected_components(pages, pagelinks, 0.05)
%time connected_components(pages, pagelinks, 0.1)

+---------+-----+------+------+
|component|count| total|  frac|
+---------+-----+------+------+
|        1|59249|118017|50.204|
|     8811|   18|118017| 0.015|
|    55824|   17|118017| 0.014|
|    79400|   16|118017| 0.014|
|    41245|   16|118017| 0.014|
+---------+-----+------+------+
only showing top 5 rows

Wall time: 26.3 s
+---------+------+------+------+
|component| count| total|  frac|
+---------+------+------+------+
|        1|212828|296197|71.854|
|   109084|    22|296197| 0.007|
|   144422|    21|296197| 0.007|
|   107609|    19|296197| 0.006|
|    98211|    18|296197| 0.006|
+---------+------+------+------+
only showing top 5 rows

Wall time: 26.9 s
+---------+------+------+------+
|component| count| total|  frac|
+---------+------+------+------+
|        1|522102|591992|88.194|
|   231613|    15|591992| 0.003|
|   188285|    14|591992| 0.002|
|    82808|    12|591992| 0.002|
|   170378|    11|591992| 0.002|
+---------+------+------+------+
only showing top 5 rows

Wall ti

In [7]:
# this takes 12 minutes to complete, with 99.996% of nodes in the largest component
# %time connected_components(pages, pagelinks, 1.0)

### Recursive bi-partitioning

In [4]:
g = sample_graph(pages, pagelinks, 0.02)
g.cache()
edges = g.edges.toPandas()

In [5]:
g.vertices.orderBy(F.asc("id")).show(n=3)
g.vertices.orderBy(F.desc("id")).show(n=3)

+---+----------+
| id|article_id|
+---+----------+
|  0|       621|
|  1|       802|
|  2|       888|
+---+----------+
only showing top 3 rows

+------+----------+
|    id|article_id|
+------+----------+
|118605|  61562569|
|118604|  61562533|
|118603|  61562477|
+------+----------+
only showing top 3 rows



In [6]:
import scipy.sparse as sp
from scipy.sparse.linalg import eigsh
from scipy.sparse.csgraph import laplacian
import numpy as np


def sparse_matrix_from_edgelist(edges):
    n = edges["src"].max()
    ones = np.ones(len(edges["dst"]))
    return sp.coo_matrix(
        (ones, (edges["src"], edges["dst"])), 
        shape=(n+1, n+1)
    ).tocsr()
    
def fiedler_vector(g):
    L = laplacian(g)
    # ordered by smallest eigenvalue
    _, v = eigsh(L, k=2)
    # fiedler vector is the second smallest eigenvalue
    return v[:,1]

In [7]:
lap = sparse_matrix_from_edgelist(edges)
%time vec = fiedler_vector(lap)
vec.shape

Wall time: 270 ms


(118606,)

In [8]:
import pandas as pd
from pyspark.sql.functions import pandas_udf, PandasUDFType

@pandas_udf("id long, value double", PandasUDFType.GROUPED_MAP)
# Input/output are both a pandas.DataFrame
def compute_fiedler(pdf):
    g = sparse_matrix_from_edgelist(pdf)
    vec = fiedler_vector(g)
    return pd.DataFrame([{"id": i, "value": v} for i, v in enumerate(vec)])

In [9]:
results = g.edges.withColumn("part_id", F.lit(0)).groupBy("part_id").apply(compute_fiedler)
%time results.show(n=5)

+---+--------------------+
| id|               value|
+---+--------------------+
|  0|-4.74153474596838...|
|  1|1.816576854708336E-7|
|  2|2.212099119240315E-7|
|  3|-2.27131145561087...|
|  4|-5.38874987399268...|
+---+--------------------+
only showing top 5 rows

Wall time: 12.7 s


In [10]:
results.selectExpr("value >= 0 as part").groupBy("part").count().show()

+-----+-----+
| part|count|
+-----+-----+
| true|58882|
|false|59724|
+-----+-----+



In [11]:
from typing import List

def induce_graph(graph, relabel=True):
    """Remove extra edges that do not belong to the graph"""
    # small dataframe for reindexing/relabeling
    rank = (
        graph.vertices
        .select("id", F.expr("row_number() over (order by id)").alias("rank"))
        # ensure 0 index for mapping into a scipy.sparse matrix
        .withColumn("rank", F.expr("rank - 1"))
    )
    vertices = graph.vertices.join(rank, on="id", how="inner")
    edges = (
        graph.edges.join(
            vertices.selectExpr("id as src", "rank as relabeled_src"),
            on="src", 
            how="inner"
        ).join(
            vertices.selectExpr("id as dst", "rank as relabeled_dst"), 
            on="dst", 
            how="inner"
        )
    )

    if relabel:
        vertices = (
            vertices
            .withColumn("original_id", F.col("id"))
            .withColumn("id", F.col("rank"))
        )
        edges = edges.selectExpr("relabeled_src as src", "relabeled_dst as dst")
    vertices = vertices.drop("rank")
    edges = edges.drop("relabeled_src").drop("relabeled_dst")

    return GraphFrame(vertices, edges)


def recursive_bipartition(graph: GraphFrame, max_iter: int = 2) -> GraphFrame:
    def undo_relabel(vertices):
        return vertices.withColumn("id", F.col("original_id")).drop("original_id")

    """
    Assumes the input graph has been relabeled, which is required for performance of
    scipy.linalg.eigsh.
    """
    def bipartition(graph: GraphFrame, partitions: List[str] = [], iteration: int = 0):
        
        partition = f"sign_{iteration}"
        fiedler_value = f"fiedler_{iteration}"
        fiedler = (
            graph
            .edges
            # necessary for collecting all edges to a single worker
            .withColumn("part", F.lit(True))
            .groupBy("part")
            .apply(compute_fiedler)
            .withColumn(partition, F.expr("value >= 0").astype("boolean"))
            .selectExpr("id", f"value as {fiedler_value}", partition)
        )
        
        # NOTE: assumes relabeling, reverse the relabeling process
        vertices = graph.vertices.join(fiedler, on="id", how="left")
        parted_graph = GraphFrame(vertices, graph.edges)
        
        if iteration == max_iter:
            return undo_relabel(parted_graph.vertices)
        else:
            positive_vertices = bipartition(
                induce_graph(
                    GraphFrame(
                        parted_graph.vertices.where(f"{partition}"),
                        parted_graph.edges,
                    ),
                    relabel=True,
                ),
                partitions + [partition],
                iteration + 1,
            )
            negative_vertices = bipartition(
                induce_graph(
                    GraphFrame(
                        parted_graph.vertices.where(f"NOT {partition}"),
                        parted_graph.edges,
                    ),
                    relabel=True,
                ),
                partitions + [partition],
                iteration + 1,
            )
            # reusing the index, should this be saved?
            return undo_relabel(
                positive_vertices
                .union(negative_vertices)
                .join(vertices.select("id", "original_id"), on="id", how="inner")
            )

    return bipartition(induce_graph(graph, relabel=True))    

In [106]:
result = recursive_bipartition(g)
%time result.show(n=2)

+---+----------+--------------------+------+--------------------+------+--------------------+------+
| id|article_id|           fiedler_0|sign_0|           fiedler_1|sign_1|           fiedler_2|sign_2|
+---+----------+--------------------+------+--------------------+------+--------------------+------+
| 12|      1915|-3.77200658910386...| false|-3.84364987273148...| false|-1.62689614330089...| false|
| 18|      2439|-2.51943472764536...| false|-8.78679236681460...| false|-8.18238274138275...| false|
| 38|      4667|-4.11143290197177E-6| false|-1.46728074632263...| false|-2.02239415597500...| false|
| 67|      7774|-8.78638934628648...| false|-3.93770260955373...| false|-1.65094696383655...| false|
| 70|      8103|8.751971287319299E-4|  true|-3.61108512861437...| false|-1.25315827966449...| false|
+---+----------+--------------------+------+--------------------+------+--------------------+------+
only showing top 5 rows

Wall time: 3min 31s


In [109]:
signs = [c for c in result.columns if c.startswith("sign_")]
result.groupBy(*signs).count().show()

+------+------+------+-----+
|sign_0|sign_1|sign_2|count|
+------+------+------+-----+
| false| false| false|15279|
|  true|  true| false|14578|
| false|  true| false|15149|
|  true| false|  true|14595|
|  true|  true|  true|14584|
|  true|  true|  null|   15|
|  true| false| false|14486|
| false| false|  true|15246|
| false|  true|  true|14983|
|  true| false|  null|    4|
| false| false|  null|    2|
+------+------+------+-----+

